<a href="https://colab.research.google.com/github/FELIX-NJUGUNA/truthscan/blob/main/Truthscan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()  # Uploading the kaggle.json file

TypeError: 'NoneType' object is not subscriptable

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d abhishekbaghel11/ai-text-detection-dataset

Dataset URL: https://www.kaggle.com/datasets/abhishekbaghel11/ai-text-detection-dataset
License(s): unknown
 94% 29.0M/30.9M [00:02<00:00, 19.0MB/s]
100% 30.9M/30.9M [00:02<00:00, 13.2MB/s]


In [ ]:
!unzip ai-text-detection-dataset.zip

Archive:  ai-text-detection-dataset.zip
  inflating: external_dataset.csv    


In [ ]:
!pip install datasets

from datasets import load_dataset

ds = load_dataset("ahmadreza13/human-vs-Ai-generated-dataset")


print(ds.head())

AttributeError: 'DatasetDict' object has no attribute 'head'

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv("/content/external_dataset.csv")

# Display the first few rows
print(df.head())

# Renove the duplicates
df = df.drop_duplicates(subset=["text"])

                                                text  generated
0  There are alot reasons to keep our the despise...          0
1  Driving smart cars that drive by themself has ...          0
2  Dear Principal,\n\nI believe that students at ...          0
3  Dear Principal,\n\nCommunity service should no...          0
4  My argument for the development of the driverl...          0


In [ ]:
import re


def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and numbers, but keep spaces
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    # Remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Assuming 'df' is your original DataFrame
df["text"] = df["text"].apply(clean_text)

# Filter out rows where 'text' is empty
df = df[df['text'] != '']

# Count the number of AI and human samples
ai_count = df[df["text"] == "ai"].shape[0]
human_count = df[df["text"] == "human"].shape[0]

# Balance the dataset
min_count = min(ai_count, human_count)
df_balanced = pd.concat([
    df[df["text"] == "ai"].sample(min_count, random_state=42),
    df[df["text"] == "human"].sample(min_count, random_state=42)
])

# Shuffle the dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

FEATURED ENGINEERING

In [ ]:
import numpy as np

# Text length
df_balanced["text_length"] = df_balanced["text"].apply(len)

# Punctuation count
df_balanced["punctuation_count"] = df_balanced["text"].apply(lambda x: sum(1 for char in x if char in [".", ",", "!", "?"]))

# Sentiment analysis (optional)
from textblob import TextBlob
df_balanced["sentiment"] = df_balanced["text"].apply(lambda x: TextBlob(x).sentiment.polarity)

COVERTING TEXT TO NUMERICAL FORMAT

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert text to TF-IDF vectors
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df_balanced["text"]).toarray()

# Combine with other features
X = np.hstack([X, df_balanced[["text_length", "punctuation_count", "sentiment"]].values])

# Labels
y = df_balanced["label"].map({"ai": 1, "human": 0}).values

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout

# Define the model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1)

In [ ]:
# Predict on the test set
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Calculate metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-Score:", f1_score(y_test, y_pred))

In [ ]:
model.save("truthscan_model.h5")

# download the model
from google.colab import files
files.download("truthscan_model.h5")